In [11]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import ArrayType, StringType, StructType, StructField
from pyspark.ml.fpm import FPGrowth # ml기반 fpgrowth는 fit으로 학습
import pyspark.sql.functions as F
import pandas as pd
#!pip install psutil
import psutil
import os
from pyspark.ml.fpm import FPGrowthModel

In [17]:
# SparkSession 초기화
spark = SparkSession.builder \
    .appName("nutr_fpgrowth") \
    .getOrCreate()

# i) 전체 모델을 딕셔너리에 저장 code1

In [18]:
def load_models(base_path):
    models = {}
    for dir_name in os.listdir(base_path):
        if ".model" in dir_name:
            gender, age_group = dir_name.split("_")[:2]
            path = os.path.join(base_path, dir_name)
            model = FPGrowthModel.load(path)
            models[(gender, age_group)] = model
    return models

In [19]:
model = load_models("./models")

In [23]:
model

{('남자',
  '0~9.model'): FPGrowthModel: uid=FPGrowth_e7f93e21dd5f, numTrainingRecords=98,
 ('남자',
  '10~19.model'): FPGrowthModel: uid=FPGrowth_a06743a4beca, numTrainingRecords=317,
 ('남자',
  '20~29.model'): FPGrowthModel: uid=FPGrowth_c4b10a2f6bf2, numTrainingRecords=321,
 ('남자',
  '30~39.model'): FPGrowthModel: uid=FPGrowth_e4dc1107ca4c, numTrainingRecords=502,
 ('남자',
  '40~49.model'): FPGrowthModel: uid=FPGrowth_53d748c1f103, numTrainingRecords=776,
 ('남자',
  '50~59.model'): FPGrowthModel: uid=FPGrowth_c21736a6a13a, numTrainingRecords=1067,
 ('남자',
  '60~69.model'): FPGrowthModel: uid=FPGrowth_982a453adf53, numTrainingRecords=997,
 ('남자',
  '70~79.model'): FPGrowthModel: uid=FPGrowth_8e23d832cab2, numTrainingRecords=258,
 ('남자',
  '80~89.model'): FPGrowthModel: uid=FPGrowth_18d58ad32501, numTrainingRecords=13,
 ('남자',
  '90~99.model'): FPGrowthModel: uid=FPGrowth_79127ca75d23, numTrainingRecords=1,
 ('여자',
  '0~9.model'): FPGrowthModel: uid=FPGrowth_c70e00a934d2, numTrainingRecords=

# ii) 단일 모델을 디렉토리에서 가져오기 code2

In [22]:
def load_model(base_path, gender, age_group):
    model_name = f"{gender}_{age_group}.model"
    model_path = os.path.join(base_path, model_name)
    return FPGrowthModel.load(model_path)

앱에서 실시간으로 제품을 추천할 때는 다음과 같은 요소들을 고려하여 결정을 내릴 수 있습니다:

응답 시간: 사용자 경험은 매우 중요합니다. 추천 모델이 실시간으로 빠르게 응답해야 사용자 만족도가 높아집니다. 만약 모델 로딩 시간이 사용자에게 눈에 띄는 지연을 일으킨다면, 모델을 미리 로드해두는 것이 좋을 수 있습니다.

사용자 동시 접속 수: 앱의 동시 사용자 수가 많다면, 각 사용자 요청마다 모델을 로드하는 것은 시스템에 큰 부담이 될 수 있습니다. 이 경우 모델을 메모리에 상주시켜두는 것이 효율적일 수 있습니다.

개인화 요구 사항: 사용자마다 개인화된 모델이 필요한 경우, 모델을 요청 시 동적으로 로드하는 전략이 더 효과적일 수 있습니다. 사용자의 최신 데이터를 반영하기 위해 모델을 주기적으로 업데이트해야 하는 상황도 이에 해당합니다.

리소스 제한: 서버의 메모리와 CPU 사용량을 체크해야 합니다. 제한된 리소스를 가진 환경에서는 모든 모델을 상주시키는 것이 비현실적일 수 있으므로, 필요할 때마다 모델을 로드하는 것이 좋을 수 있습니다.

인프라 및 캐싱 전략: 앱 서버의 인프라가 스케일 아웃(scaling out)을 지원한다면, 캐싱 서비스를 사용하여 자주 사용되는 모델을 캐시에 저장하고, 필요에 따라 캐시에서 모델을 빠르게 가져오는 방식이 가능합니다.

추천 전략:
미리 로딩: 메모리와 CPU 리소스가 충분하고, 사용자별로 크게 다른 모델을 사용하지 않는다면, 앱 시작 시 모든 모델을 메모리에 로드해 두는 전략을 사용합니다.
동적 로딩: 사용자별로 맞춤 모델이 필요하거나, 리소스가 제한적이라면, 각 사용자의 요청에 맞추어 모델을 동적으로 로드합니다.
하이브리드 접근: 가장 자주 사용되는 몇 가지 모델은 미리 로드해 두고, 나머지는 동적으로 로드하는 하이브리드 접근 방법을 취할 수도 있습니다.
마지막으로, 앱의 성능을 모니터링하고 사용자의 행동을 분석하여 로딩 전략을 지속적으로 최적화하는 것이 중요합니다. 시간이 지남에 따라 사용자의 행동이 변하고, 앱의 사용량이 변할 수 있으므로, 유연하게 접근하여 필요에 따라 전략을 조정해야 합니다.